In [11]:

import pandas as pd       
import openpyxl       
from selenium import webdriver       
from selenium.webdriver.common.keys import Keys       
from selenium.webdriver import ActionChains       
from bs4 import BeautifulSoup       
from tkinter import filedialog, Tk       
import time       
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC       
from selenium.webdriver.common.by import By  
from selenium import webdriver       
import os       
from tqdm import tqdm       
from selenium.common.exceptions import TimeoutException       
from selenium.webdriver.chrome.options import Options       
import matplotlib.font_manager as fm  
import matplotlib.pyplot as plt  
import datetime       
from selenium.webdriver.chrome.service import Service    
from selenium.common.exceptions import UnexpectedAlertPresentException  


def naver(soup) :      
    try :      
        seller = soup.find("span", attrs={"class": "KasFrJs3SA"}).get_text()      
        uploaded_price = soup.find("span", attrs={'class' :'_1LY7DqCnwR' }).get_text()      
        platform = "스마트스토어"       
    except AttributeError :     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "스마트스토어"         
    return seller, uploaded_price, platform     
def coupang(soup) :     
    try :      
        seller = soup.find("a", attrs={"class": "prod-sale-vendor-name"}).get_text()      
        uploaded_price = soup.find("span", attrs={'class' :'total-price' }).get_text()      
        platform = "쿠팡"            
    except AttributeError :     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "쿠팡"     
    return seller, uploaded_price, platform       
       
def interpark(soup):           
    time.sleep(5)     
    try:     
        seller = soup.find("div", attrs={"class": "sellerName"}).get_text()     
        uploaded_price = soup.find("span", attrs={'class': 'priceWrap'}).get_text()     
        platform = "인터파크"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "인터파크"     
    return seller, uploaded_price, platform     
def auction(soup) :     
    try :     
        seller = soup.find("a", attrs={"class": "link__seller sp_vipgroup--before sp_vipgroup--after"}).get_text()       
        uploaded_price = soup.find("strong", attrs={"class" :"price_real" }).get_text()      
        platform = "옥션/G마켓"      
    except AttributeError :     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "옥션/G마켓"     
    return seller, uploaded_price, platform    
      
def gmarket(soup) :     
    try:     
        seller = soup.find("a", attrs={"class": "link__seller sp_vipgroup--before sp_vipgroup--after"}).get_text()     
        uploaded_price = soup.find("strong", attrs={'class': 'price_real'}).get_text()     
        platform = "G마켓/옥션"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "G마켓/옥션"     
    return seller, uploaded_price, platform     
def ssg(soup):     
    try:     
        seller = soup.find("a", attrs={"class": "cdtl_info_tit_link"}).get_text()     
        uploaded_price = soup.find("em", attrs={'class': 'ssg_price'}).get_text()     
        platform = "SSG"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "SSG닷컴"     
    return seller, uploaded_price, platform     
def elevenst(soup):     
    try:     
        seller = soup.find("h1", attrs={"class": "c_product_store_title"}).find('a').get_text().strip()     
        uploaded_price = soup.find("span", attrs={'class': 'num value'}).get_text()     
        platform = "11번가"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "11번가"     
    return seller, uploaded_price, platform    
      
def wemakeprice(soup):     
    try:     
        seller = soup.find("span", attrs={"class": "store_name"}).get_text()     
        uploaded_price = soup.find("em", attrs={'class': 'num'}).get_text()     
        platform = "위메프"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "위메프"     
    return seller, uploaded_price, platform     
def lotte(soup):     
    try:     
        seller = soup.find("div", attrs={"class": "top"}).get_text()     
        uploaded_price = soup.find("span", attrs={'class': 'won'}).get_text().strip()     
        platform = "롯데ON"     
    except AttributeError:     
        seller = "확인필요"     
        uploaded_price = "확인필요"     
        platform = "롯데ON"     
    return seller, uploaded_price, platform     
def unknown(soup):     
    seller = "*확인필요*"     
    uploaded_price = ""     
    platform = ""     
    return seller, uploaded_price, platform     
    
            
    
    
dfs = []
            
# 엑셀 파일 읽어오기       
file_path = filedialog.askopenfilename(title="가격지도 파일", defaultextension=".xlsx")       
df_input = pd.read_excel(file_path, header =0)             
            
options = Options()       
options = webdriver.ChromeOptions()       
driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)       
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})       
        
         
for i in tqdm(range(len(df_input))):       
    # 검색할 키워드를 주문번호로 대체하여 url 생성       
    keyword = str(df_input.loc[i, '주문번호'])       
    minimum_price = int(df_input.loc[i, '최소값'])       
    maximum_price = int(df_input.loc[i, '최대값'])   
    filter = str(df_input.loc[i, '필터'])    
                
    #시트에서 지도가 추출하기       
    comsmart_standard = int(df_input.loc[i, '지도가'])       
                
    url = f'https://search.shopping.naver.com/search/all?origQuery={keyword}&pagingIndex=1&pagingSize=80&productSet=total&query={keyword}&sort=rel&timestamp=&viewType=list'     
                
    driver.get(url)       
    time.sleep(2)       
                
    search_btns = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[data-testid='SEARCH_SUB_FILTER_ORDER_LIST']")))       
                
    for btn in search_btns:       
        if btn.text.strip() == "낮은 가격순":       
            btn.click()       
            break       
                
    #최적화 필터 제거하기           
    next_link = WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.subFilter_btn_radio__13PEL')))       
    if 'on' in next_link.get_attribute('class'):       
        next_link.click()       
                    
    time.sleep(3)       
                    
            
    min_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@title='최소가격 입력']")))     
    min_price.clear()     
    min_price.send_keys(minimum_price)     
        
    time.sleep(1)     
        
    st_date = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@title='최대가격 입력']")))     
    st_date.clear()      
    st_date.send_keys(maximum_price)      
        
    time.sleep(1)     
        
    search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='filter_price_srh__D6arg' and @role='button']")))        
    search_button.click()     
        
    time.sleep(1)     
                
    for c in range(0, 20):       
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/20*" + str(c+1) + ");")       
            time.sleep(0.1)       
                    
    time.sleep(3)    

    des_list = []
    link_list = []
    seller_list = []
    prc_list = []
    gap_list = []
    comment = []
    platform_list = []  
    
    html = driver.page_source       
    soup = BeautifulSoup(html, 'html.parser')        
    flatform = ""
    image_list = soup.select("div.product_item__MDtDF")        
            
    for item in image_list:
        for des in item.select("a.product_link__TrAac.linkAnchor"):
            des_text = des['title']
            des_list.append(des_text)
            link_text = des['href']
            link_list.append(link_text)
    for item in image_list:
        for seller in item.select_one("div.product_mall_title__Xer1m > a"):
            seller_text = seller.get_text()
            seller_list.append(seller_text)
    for item in image_list:        
        for price in item.select("span.price_num__S2p_v"):
            prc = int(price.get_text().replace("원", "").replace(",", ""))
            prc_list.append(prc)
            gap = comsmart_standard - prc
            gap_list.append(gap)
            comment_text = "안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 " + str(comsmart_standard) + "원으로 현재 업로드하신 금액과는 당사 지도가 대비" + str(gap) + "원 차이가 있으니 판매 단가 수정을 부탁드립니다." 
            comment.append(comment_text)  

    df = pd.DataFrame({  
        "모델명" : [keyword]*len(des_list),               
        "상세정보": des_list,        
        "지도가" : [comsmart_standard]*len(des_list),          
        "업체등록가": prc_list,      
        "가격차이": gap_list,  
        "판매처" : seller_list,  
        "플랫폼" : [flatform]*len(des_list),
        "링크주소": link_list,
        "안내문구" : comment,   
        })     
    
    if filter :
        df = df[~df['상세정보'].str.contains(filter)]
    
    dfs.append(df.copy())
df_total = pd.concat(dfs, ignore_index=True)
df_total = df_total[ df_total['판매처'] != '쇼핑몰별 최저가' ]
ddf_total = df_total[df_total['가격차이'] >= 1].reset_index(drop=True)


C:\Users\gidum\AppData\Local\Temp\ipykernel_5480\4000004401.py:135: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)
100%|██████████| 4/4 [01:24<00:00, 21.21s/it]


In [15]:
ddf_total

,모델명,상세정보,지도가,업체등록가,가격차이,판매처,플랫폼,링크주소,안내문구
0,UD887,on/ 휴대용 LED 다기능 스마트전자 줄자 UD887,31450,31440,10,,,https://cr.shopping.naver.com/adcr.nhn?x=WyeEO...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3145...
1,VK363,Coms Coms VK363 2포트 HDMI KVM 스위치/2대 연결/HDMI 2.0,54400,53910,490,,,https://cr.shopping.naver.com/adcr.nhn?x=qL8JU...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 5440...
2,VK363,COMS Coms VK363 2포트 HDMI KVM 스위치/2대 연결/HDMI 2.0,54400,54210,190,,,https://cr.shopping.naver.com/adcr.nhn?x=BtpYg...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 5440...
3,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,359720,31280,,,https://cr.shopping.naver.com/adcr.nhn?x=QWxE4...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
4,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,359720,31280,,,https://cr.shopping.naver.com/adcr.nhn?x=qs7hS...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
5,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,359720,31280,,,https://cr.shopping.naver.com/adcr.nhn?x=Fn4s3...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
6,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,359720,31280,,,https://cr.shopping.naver.com/adcr.nhn?x=vh5TR...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
7,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,359720,31280,,,https://cr.shopping.naver.com/adcr.nhn?x=qbyWL...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
8,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,359720,31280,,,https://cr.shopping.naver.com/adcr.nhn?x=h6XXL...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
9,WN004S,WN004S Coms WN004S 8채널 NVR 500백만 화소IP,391000,390000,1000,,,https://cr.shopping.naver.com/adcr.nhn?x=CG7oG...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 3910...
